In [1]:
"""
当时间步数较大或者时间步较小时，循环神经网络的梯度较容易出现衰减或爆炸。
门控循环神经网络（gated recurrent neural network）的提出，正是为了更好地捕捉时间序列中时间步距离较大的依赖关系。
它通过可以学习的门来控制信息的流动。
它引入了重置门（reset gate）和更新门（update gate）的概念,他们的激活函数都是sigmoid
重置门Rt和更新门Zt中每个元素的值域都是[0,1]
"""

'\n当时间步数较大或者时间步较小时，循环神经网络的梯度较容易出现衰减或爆炸。\n门控循环神经网络（gated recurrent neural network）的提出，正是为了更好地捕捉时间序列中时间步距离较大的依赖关系。\n它通过可以学习的门来控制信息的流动。\n它引入了重置门（reset gate）和更新门（update gate）的概念,他们的激活函数都是sigmoid\n重置门Rt和更新门Zt中每个元素的值域都是[0,1]\n'

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import backend as f
import numpy as np
import math
import sys
import time
sys.path.append("..") 
import d2lzh_tensorflow2 as d2l

In [2]:
(corpus_indices, char_to_idx, idx_to_char, vocab_size) = d2l.load_data_jay_lyrics(
    "../data/jaychou_lyrics.txt.zip")

In [3]:
num_hiddens = 256
rnn_layer = keras.layers.GRU(num_hiddens,
                             time_major=True, # 含义在doc中没找到,其含义可能是
                             return_sequences=True,
                             return_state=True # state是每个rnn单元输出的隐藏层状态，用于下一个单元的输入
                            )

In [4]:
cell=keras.layers.GRUCell(num_hiddens,kernel_initializer='glorot_uniform')
batch_size = 2
state = cell.get_initial_state(batch_size=batch_size,dtype=tf.float32) # 初始化的隐藏状态列表
state.shape

TensorShape([2, 256])

In [5]:
num_steps = 23
X = tf.random.uniform(shape=(num_steps, batch_size, vocab_size)) # num_steps是使用多少个RNNcell来构造模型
Y, state_new = rnn_layer(X, state)  # 因为batch_size是2，所以state_new长度是2
Y.shape, len(state_new), state_new[0].shape, state_new[1].shape
# 注意 Y的维度是(num_steps, batch_size, num_hidden) state_new的维度是 (batch_size, num_hidden)

(TensorShape([23, 2, 256]), 2, TensorShape([256]), TensorShape([256]))

In [6]:
# 没训练之前，输出预测歌词的结果
model = d2l.RNNModel(rnn_layer, vocab_size)
d2l.predict_rnn_keras('分开', 10, model, vocab_size,  idx_to_char, char_to_idx)

'分开谢绕枪怪剧瘦幽迎悲檐'

In [14]:
# 训练多轮，每隔N轮输出预测结果
num_epochs, batch_size, lr, clipping_theta = 250, 32, 1e2, 1e-2
pred_period, pred_len, prefixes = 50, 50, ['分开', '不分开']
d2l.train_and_predict_rnn_keras(model, num_hiddens, vocab_size, 
                            corpus_indices, idx_to_char, char_to_idx,
                            num_epochs, num_steps, lr, clipping_theta,
                            batch_size, pred_period, pred_len, prefixes)

epoch 50, perplexity 35.388141, time 2.92 sec
 - 分开                                                  
 - 不分开                                                  
epoch 100, perplexity 121.625929, time 2.86 sec
 - 分开透透透透透透透透透透透透透透透透透透透透透透透透透透透透透透透透透透透透透透透透透透透透透透透透透透
 - 不分开透透透透透透透透透透透透透透透透透透透透透透透透透透透透透透透透透透透透透透透透透透透透透透透透透透
epoch 150, perplexity 16927.761040, time 2.83 sec
 - 分开腮oo理葛广葛彩葛葛田仔仔红夜仔仔仔红夜理裂缝缝器理葛仔藤仔仔红夜理葛仔理葛葛广葛仔仔凉藤理葛仔藤仔
 - 不分开镇蜘仔仔红夜理葛御理葛仔藤仔理葛葛广葛仔仔红夜理裂缝缝器理葛彩葛葛广葛仔仔红夜理裂缝缝器理葛彩葛葛广
epoch 200, perplexity 1892.750020, time 2.85 sec
 - 分开悉楷楷仔悉括楷楷楷仔悉楷括楷悉楷仔括楷楷把悉楷悉楷仔括楷楷悉楷悉楷仔括括楷悉楷悉楷楷仔括括楷悉楷悉楷
 - 不分开括楷楷悉悉楷楷楷仔悉括楷悉楷仔括楷楷把悉楷悉楷仔括楷楷悉楷悉楷仔括括楷悉楷悉楷楷仔括括楷悉楷悉楷楷仔
epoch 250, perplexity 720.831429, time 2.83 sec
 - 分开藤把把把悉悉悉楷藤藤藤藤藤藤藤藤藤藤藤藤藤藤藤藤藤藤藤藤藤藤藤藤藤藤藤藤藤藤藤藤藤藤藤藤藤藤藤藤藤藤
 - 不分开镇把把藤悉悉藤藤藤藤藤藤藤藤藤藤藤藤藤藤藤藤藤藤藤藤藤藤藤藤藤藤藤藤藤藤藤藤藤藤藤藤藤藤藤藤藤藤藤藤


In [ ]:
"""
训练效果略有改善，但还是不够好
"""